In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import requests
from datetime import datetime
from citipy import citipy
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the lats and longs to a list
coordinates = list(lat_lngs)
print(coordinates)

[(23.014114215287833, -118.64099537128416), (-55.87785732391645, 120.35133306629916), (5.027420349224428, 16.14946603939228), (-0.3100530273153339, 25.993655262956906), (-30.223994358002493, 30.936800148591175), (11.84699357829409, -143.92153957351772), (-60.61316788800373, -51.81345426605657), (78.88929084935714, -82.52894351547631), (81.54744108187, 87.70360469738597), (-22.618665202135332, -120.11810463942686), (72.81899002111666, 97.07010328616417), (28.915377879123454, -119.00246899527285), (2.535687077572362, 91.81938670812877), (-1.3456458609579869, 16.399802754891425), (-88.84506274897021, -143.67717029684863), (-71.19569770794075, 41.02827152005449), (72.0720065138959, -28.023261493890686), (54.373843097973435, 155.4530521518393), (71.74214223949951, 34.87103921457725), (64.82341700432448, -76.6251064285028), (-70.95725699697249, 41.966725635915424), (-24.979125916557308, -1.0013960650357205), (48.33844233569786, 20.466802095940636), (27.039289552722337, -53.91534303933339), (

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

756

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6dad2c9df2c03a85596c79c0e3847d92


In [7]:
# Create an empty list to hold weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Set counter
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Descritption": weather_description,
                          "Date": city_date})
        
# If error, skip city
    except:
        print("City not found. Skipping...")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | guerrero negro
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | carnot
Processing Record 4 of Set 1 | kisangani
Processing Record 5 of Set 1 | umkomaas
Processing Record 6 of Set 1 | hilo
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | qaanaaq
Processing Record 9 of Set 1 | dikson
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | khatanga
Processing Record 12 of Set 1 | san quintin
Processing Record 13 of Set 1 | meulaboh
Processing Record 14 of Set 1 | gamboma
Processing Record 15 of Set 1 | mataura
Processing Record 16 of Set 1 | port alfred
Processing Record 17 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 18 of Set 1 | sobolevo
Processing Record 19 of Set 1 | polyarnyy
Processing Record 20 of Set 1 | iqaluit
Processing Record 21 of Set 1 | jamestown
Processing Record 22 of Set 1 | putnok
Processing Rec

In [8]:
# Convert the arraay of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Descritption,Date
0,Guerrero Negro,27.9769,-114.0611,51.03,41,0,2.46,MX,clear sky,2022-02-04 14:30:49
1,Albany,42.6001,-73.9662,28.27,86,100,3.00,US,light snow,2022-02-04 14:35:04
2,Carnot,4.9409,15.8665,94.73,12,100,5.91,CF,overcast clouds,2022-02-04 14:35:25
3,Kisangani,0.5167,25.2000,82.76,69,75,4.61,CD,broken clouds,2022-02-04 14:35:26
4,Umkomaas,-30.2067,30.7978,72.68,87,100,17.11,ZA,light rain,2022-02-04 14:35:26
5,Hilo,19.7297,-155.0900,67.12,75,0,5.75,US,clear sky,2022-02-04 14:35:26
6,Ushuaia,-54.8000,-68.3000,47.86,49,20,10.36,AR,few clouds,2022-02-04 14:35:27
7,Qaanaaq,77.4840,-69.3632,-7.49,60,100,8.86,GL,overcast clouds,2022-02-04 14:35:27
8,Dikson,73.5069,80.5464,-2.13,90,100,5.17,RU,overcast clouds,2022-02-04 14:35:27
9,Rikitea,-23.1203,-134.9692,77.77,73,100,12.08,PF,overcast clouds,2022-02-04 14:35:28


In [12]:
# Create the csv file
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export city_data into the csv
city_data_df.to_csv(output_data_file, index_label="City")